In [1]:
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
fig_DIR = "../figs/pbp_related/"

In [27]:
import os
os.environ["R_HOME"] = "C:\\Program Files\\R\\R-4.3.2\\"
%reload_ext rpy2.ipython

c:\Users\pansr\Documents\NBA\.venv\lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [5]:
url = "https://api.pbpstats.com/get-game-logs/nba"
params = {
    # "Season": "2019,2020,2021,2022,2023", # To get for multiple seasons, separate seasons by comma
    "Season": "2023-24",
    "SeasonType": "Regular Season",
    "EntityId": "201566",
    "EntityType": "Player" # Use Opponent to get opponent stats
}
response = requests.get(url, params=params)
response_json = response.json()
totals = response_json['single_row_table_data']
game_logs = response_json['multi_row_table_data']

In [44]:
seasons = ["2016-17","2017-18","2018-19","2019-20","2020-21","2021-22","2022-23","2023-24"]
Date, Usage, Team = [],[],[]
for season in seasons:
    params["Season"]=season
    response = requests.get(url, params=params)
    response_json = response.json()
    totals = response_json['single_row_table_data']
    game_logs = response_json['multi_row_table_data']
    Dates = [game["Date"] for game in game_logs]
    Usages = [game["Usage"] for game in game_logs]
    Teams = [game["Team"] for game in game_logs]
    Date += Dates
    Usage += Usages
    Team +=Teams

In [62]:
df

,index,Date,Team,Usage,Games
0,0,2016-10-26,OKC,31.818182,1
1,1,2016-10-28,OKC,50.000000,2
2,2,2016-10-30,OKC,35.294118,3
3,3,2016-11-02,OKC,54.320988,4
4,4,2016-11-03,OKC,38.028169,5
...,...,...,...,...,...
508,508,2023-10-27,LAC,11.250000,509
509,509,2023-10-29,LAC,25.806452,510
510,510,2023-10-31,LAC,24.691358,511
511,511,2023-11-01,LAC,28.155340,512


In [65]:
df = pd.DataFrame({"Date":Date,"Team":Team,"Usage":Usage})
df = df[df["Team"] != "OKC" ] 
df.reset_index(drop=False,inplace=True)
df["Games"]=df["index"]+1

In [82]:
%%R -i df -w 5 -h 4 --units in -r 150
library(ggplot2)
library(dplyr)
library(nbastatR)

# custom ggplot2 theme
theme_owen <- function () { 
  theme_minimal(base_size=9, base_family="Consolas") %+replace% 
    theme(
      panel.grid.minor = element_blank(),
      plot.background = element_rect(fill = 'floralwhite', color = "floralwhite")
    )
}

df$Date <- as.Date(df$Date)
df$Team <- factor(df$Team, levels = c("HOU", "WAS", "LAL","LAC"))
# Most basic bubble plot
p <- ggplot(df, aes(x=Games, y=Usage,color=Team)) +
  # geom_line() + 
  geom_point() +
  geom_smooth(method="loess") +
  # geom_text(aes(label=Team), size=3) +
  geom_hline(yintercept=30, linetype="dashed", color="black", size=0.5) +
  scale_color_manual(values = c("#CE1141", "#002B5C", "#552582","#006BB6")) +
  theme_owen() +
  theme(plot.title.position = 'plot', 
        plot.title = element_text(face = 'bold'), 
        plot.margin = margin(10, 10, 15, 10)) +
  theme(axis.text.x = element_text(size=10, face="bold", color = "black"),
        axis.text.y = element_text(size=10, face="bold", color = "black"),
        axis.title.x = element_text(size=12, face="bold", colour = "black"),
        axis.title.y = element_text(size=12, face="bold", colour = "black")) +
  labs(x = "Games Played Since Houston", 
       y = "Usage", 
       title = "Russell Westbrook Usage Trends Since Houston",
       subtitle = "Was Westbrook not given a chance to be productive with the Lakers?")

ggsave("../figs/R/Russ_Usage_Trends.png", p, w = 8, h = 5, dpi = 600)

`geom_smooth()` using formula = 'y ~ x'
